In [ ]:
#@title ドライブをマウント 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title データを保存するディレクトリのパスを指定
#@markdown ランタイムに接続しドライブを後、コピーしてくる<br>
#@markdown 後ろの / は追加するので、追記は不要<br>

cwd = "/content/drive/MyDrive/RNAseq/Outputs/Hypoxia2" #@param {type:"string"}
cwd = cwd + "/"

# 必要なモジュールをインポートする
import pandas as pd
import numpy as np
import urllib.parse
from google.colab import files

# カレントディレクトリの変更
import os
os.makedirs(cwd , exist_ok = True)
os.chdir(cwd)
currentdir = os.getcwd()

In [ ]:
#@title 実験名とN数を設定
#@markdown 保存ファイル名の前に付ける名前を設定する<br>
#@markdown N数は1と3のみ実装、他の数の場合は分けて適宜使用<br>
exp_name = "DC_only" #@param {type:"string"}
exp_name = exp_name + "_"
n_number = "3" #@param ["1", "3"]

In [ ]:
#@title 各ファイルのパスを指定
#@markdown N=1の場合は、カウントデータ2, 3を空白にする

#@markdown コントロール群のカウントデータ(N=1, N=3の1つ目)
count_Cont1 = "/content/drive/MyDrive/RNAseq/Inputs/Hypoxia2/F45_NRX_DC.txt" #@param {type:"string"}
#@markdown コントロール群のカウントデータ2(N=3の2つ目)
count_Cont2 = "/content/drive/MyDrive/RNAseq/Inputs/Hypoxia2/M21_NRX_DC.txt" #@param {type:"string"}
#@markdown コントロール群のカウントデータ3(N=3の3つ目)
count_Cont3 = "/content/drive/MyDrive/RNAseq/Inputs/Hypoxia2/M22_NRX_DC.txt" #@param {type:"string"}

#@markdown テスト群のカウントデータ1(N=1, N=3の1つ目)
count_Test1 = "/content/drive/MyDrive/RNAseq/Inputs/Hypoxia2/F45_HYP_DC.txt" #@param {type:"string"}
#@markdown テスト群のカウントデータ2(N=3の2つ目)
count_Test2 = "/content/drive/MyDrive/RNAseq/Inputs/Hypoxia2/M21_HYP_DC.txt" #@param {type:"string"}
#@markdown テスト群のカウントデータ3(N=3の3つ目)
count_Test3 = "/content/drive/MyDrive/RNAseq/Inputs/Hypoxia2/M22_HYP_DC.txt" #@param {type:"string"}

#@markdown アノテーションファイル
gtf_path = "/content/drive/MyDrive/RNAseq/SourceFiles/Homo_sapiens.GRCh38.94.gtf" #@param {type:"string"}

In [ ]:
#@title featurecountsの出力名が長いので、サンプル名をわかりやすく変更する
#@markdown 以下に、変更後の名前を指定する
Cont_name = 'N_DC' #@param {type:"string"}
Test_name = 'H_DC' #@param {type:"string"}

ここまでを入力後、「ランタイム」タブ→「すべてのセルを実行」を選択(ショートカット：Ctrl + F9)<br>
データはGoogleドライブ上に保存される

===================================================================
============================================

In [ ]:
#@title gene_idとproductの対応表作成
#@markdown アノテーションファイルから、Gene_idとproduct(Gene_name)の対応表を作成する<br>ファイルの形式によっては、コードを多少書き換える必要である場合もある<br>ensemblの出してるgtf, gffファイルならこのコードでいけるはず

# 出力用のリストを定義
outputlist = []
# 出力用のcsvを作成
fout = open(exp_name + '_id_name.csv', 'w')
# gtfファイルを開く
with open(gtf_path, 'r') as fin:
    # gtfデータを1行ずつ読み込む
    for line in fin:
      # 出力用の変数を定義
        product = ''
        geneid = ''
        
        # 行末の空白を削除
        line=line.rstrip()
        # #で始まっているデータ行を除く
        if not line.startswith('#'):
            # gtfをタブ区切りで認識し、その9列目(1列目が0なので8)を読む
            s=line.split('\t')
            items=s[8].split(';')
            
            # 各行の9列目について
            for item in items:
                # ' gene_name "'で始まる要素なら、"で分割して=の右側を取出し、url逆変換する
                if item.startswith(' gene_name "'):
                    product = item.split('"')[1]
                    product = urllib.parse.unquote(product)

                # 'gene_id "'で始まる要素なら、"で分割して右側を取出す
                if item.startswith('gene_id "'):
                    geneid = item.split('"')[1]

                # gene_name と gene_idが両方あれば、出力リストに取り込む
                if (geneid != '') and (product != ''):
                    outputlist.append(geneid+','+product)

# 重複を除去、並び順をソートの上で、ヘッダーを追加する
outputlist = sorted(list(set(outputlist)))
outputlist.insert(0 , "Geneid"+','+"product")
  
# csvに書き込んでいく
fout = open(exp_name + "id_name.csv", "w")
for u in outputlist:
    print(u, file=fout)
fout.close()

# csvをData frameとして認識後、ヘッダーを除去して再保存する
df_id_name = pd.read_csv(exp_name + '_id_name.csv')
df_id_name.to_csv(exp_name + '_id_name.csv' ,sep = ',' , index=False , header=False) 

# 対応表のData Frameを表示する

# print(df_id_name)

===================================================================
============================================

In [ ]:
#@title カウントデータの調整
#@markdown カウントデータから必要な部分を抽出し、Gene_idを元に結合する<br>必要な部分：Geneid , Length , コントロール群のカウントデータ , サンプル群のカウントデータ

# N=1の処理
if n_number == "1":
  # データの読み込み
  df_Cont1 = pd.read_table(count_Cont1 , index_col = 0 , skiprows = 1)
  df_Test1 = pd.read_table(count_Test1 , index_col = 0 , skiprows = 1)

  # TPMの計算につかわない列を削除
  drop_col = ['Chr' , 'Start' , 'End' , 'Strand']
  df_Cont1 = df_Cont1.drop(drop_col, axis=1)

  drop_col = ['Chr' , 'Start' , 'End' , 'Strand' , 'Length']
  df_Test1 = df_Test1.drop(drop_col, axis=1)

  # 表をくっつける、名前の変更
  df_Merge = pd.merge(df_Cont1 , df_Test1, on='Geneid')
  df_Merge.columns = ['Length',
                      Cont_name ,
                      Test_name]

# N=3の処理
elif n_number == "3":
    # データの読み込み
    df_Cont1 = pd.read_table(count_Cont1 , index_col = 0 , skiprows = 1)
    df_Cont2 = pd.read_table(count_Cont2 , index_col = 0 , skiprows = 1)
    df_Cont3 = pd.read_table(count_Cont3 , index_col = 0 , skiprows = 1)
    df_Test1 = pd.read_table(count_Test1 , index_col = 0 , skiprows = 1)
    df_Test2 = pd.read_table(count_Test2 , index_col = 0 , skiprows = 1)
    df_Test3 = pd.read_table(count_Test3 , index_col = 0 , skiprows = 1)

    # TPMの計算につかわない列を削除
    drop_col = ['Chr' , 'Start' , 'End' , 'Strand']
    df_Cont1 = df_Cont1.drop(drop_col, axis=1)

    drop_col = ['Chr' , 'Start' , 'End' , 'Strand' , 'Length']
    df_Cont2 = df_Cont2.drop(drop_col, axis=1)
    df_Cont3 = df_Cont3.drop(drop_col, axis=1)
    df_Test1 = df_Test1.drop(drop_col, axis=1)
    df_Test2 = df_Test2.drop(drop_col, axis=1)
    df_Test3 = df_Test3.drop(drop_col, axis=1)

    # 表をくっつける、名前の変更
    df_Merge = pd.merge(df_Cont1, df_Cont2, on='Geneid')
    df_Merge = pd.merge(df_Merge, df_Cont3, on='Geneid')
    df_Merge = pd.merge(df_Merge, df_Test1, on='Geneid')
    df_Merge = pd.merge(df_Merge, df_Test2, on='Geneid')
    df_Merge = pd.merge(df_Merge, df_Test3, on='Geneid')

    df_Merge.columns = ['Length',
                        Cont_name + "_1" ,
                        Cont_name + "_2" ,
                        Cont_name + "_3" ,
                        Test_name + "_1" ,
                        Test_name + "_2" ,
                        Test_name + "_3"]

#以下共通、出力する
# df_Merge.to_csv(exp_name + "countdata.csv")
# print(df_Merge)

In [ ]:
#@title 調整したカウントデータとid_nameの結合
#@markdown Gene_idを元に、カウントデータにGene_nameを結合する

# merge(左側のDataFrame, 右側のDataFrame , on=どの行を見るか)で結合
df_with_product = pd.merge(df_id_name , df_Merge , on='Geneid')

# インデックス行を指定
df_with_product = df_with_product.set_index(["Geneid" , "product"])

print(df_with_product)

                            Length  N_DC_1  N_DC_2  ...  H_DC_1  H_DC_2  H_DC_3
Geneid          product                             ...                        
ENSG00000000003 TSPAN6        4535     719     594  ...     635     612     804
ENSG00000000005 TNMD          1610       0       0  ...       0       0       0
ENSG00000000419 DPM1          1207     686     612  ...     725     796     817
ENSG00000000457 SCYL3         6883     209     179  ...     182     248     200
ENSG00000000460 C1orf112      5967      86     163  ...      67      98      77
...                            ...     ...     ...  ...     ...     ...     ...
ENSG00000285990 AL589743.7     647       0       0  ...       0       0       0
ENSG00000285991 AL355312.5    5065       1       0  ...       0       0       0
ENSG00000285992 AC120036.5     956       0       0  ...       0       0       0
ENSG00000285993 AC018931.1    1246       0       0  ...       0       0       0
ENSG00000285994 AL731559.1    3732      

===================================================================
============================================

In [ ]:
#@title iDEP用のidとカウントを対応させたcsvを出力させる
df_iDEP = df_with_product.reset_index()
df_iDEP = df_iDEP.set_index("Geneid")

df_iDEP = df_iDEP.drop(["product" , "Length"] , axis = 1)
df_iDEP.to_csv(exp_name + "for_iDEP.csv", sep=",")

===================================================================
============================================

In [ ]:
#@title TPMの計算式を定義

# 計算式を定義しているだけ

def normalize_per_million_reads(df):
    # RPM/FPM = reads per million  fragments per million
    sum = df.sum()
    return 10**6 * df / sum    # 列ごとにカウント/総リード数計算

def normalize_per_kilobase(df, gene_length):
    # FPKM = fragments per kilobase of exon per million reads mapped
    ret = ((df.copy().T) * 10**3 / gene_length).T
    return ret

def normalize_tpm(df, gene_length):
    # TPM = transcripts per kilobase million 
    # normalize_per_kilobaseの結果にnormalize_per_millionを施す
    df_tmp = df.copy()
    df_tmp = normalize_per_kilobase(df_tmp, gene_length)
    df_tmp = normalize_per_million_reads(df_tmp)
    return df_tmp

In [ ]:
#@title カウントデータをTPM正規化

if n_number == "1":
  df_count = df_with_product.iloc[: , 1:3]

if n_number == "3":
  df_count = df_with_product.iloc[: , 1:7]


gene_length = df_with_product['Length'] 

# TPMによる正規化
df_tpm = normalize_tpm(df_count, gene_length)
df_tpm.to_csv(exp_name + "tpm.csv", sep=",")

# print(df_tpm)

===================================================================
============================================

出力ファイル<br>
・実験名_id_name.csv
　　　　→Gene_idとProduct(遺伝子名)との対応表<br>
・実験名_for_iDEP.csv
　　　　→Gene_idとカウント数との対応表<br>
・実験名_tpm.csv
　　　　　　→遺伝子名とTPM値を対応付けたcsvファイル